# TabNet Model Test

In this notebook, we'll test a training loop for the TabNet model 


In [1]:
import sys
sys.path.append('../src')

from models.lib.neural import *
from models.lib.data import *
from models.lib.train import *

import helper 
from helper import gene_intersection
from pytorch_tabnet.tab_network import TabNet

import torch.nn as nn 
import torch.optim as optim
import torch
from tqdm import tqdm
from torch.utils.data import Subset
from helper import seed_everything

seed_everything(42)
t = helper.INTERIM_DATA_AND_LABEL_FILES_LIST
datafiles, labelfiles = zip(*t.items())
datafiles = [f'../data/interim/{f}' for f in datafiles]
labelfiles = [f'../data/processed/labels/{f}' for f in labelfiles]
refgenes = gene_intersection()

datafiles, labelfiles

(['../data/interim/primary_bhaduri_T.csv',
  '../data/interim/allen_cortex_T.csv',
  '../data/interim/allen_m1_region_T.csv',
  '../data/interim/whole_brain_bhaduri_T.csv'],
 ['../data/processed/labels/primary_bhaduri_labels.csv',
  '../data/processed/labels/allen_cortex_labels.csv',
  '../data/processed/labels/allen_m1_region_labels.csv',
  '../data/processed/labels/whole_brain_bhaduri_labels.csv'])

In [2]:
import pytorch_lightning

First, we'll define our train, val and test sets, then generate the associated DataLoaders and try training.

In [3]:
trainloaders, valloaders, testloaders = generate_loaders(
    datafiles=datafiles,
    labelfiles=labelfiles,
    class_label='Type',
    refgenes=refgenes,
    skip=3
)

trainloaders

In [4]:
trainloaders[0]

In [5]:
train, val, test = generate_single_dataset(
    datafiles[0],
    labelfiles[0],
    'Type',
    skip=3,
)

In [6]:
# print(len(train[0][0]))

In [7]:
# trainloader_map, _, _ = generate_single_dataloader(
#     datafile=datafiles[0], 
#     labelfile=labelfiles[0], 
#     class_label='Type',
#     skip=3,
#     map_genes=True
# )

# trainloader_nomap, _, _ = generate_single_dataloader(
#     datafile=datafiles[0], 
#     labelfile=labelfiles[0], 
#     class_label='Type',
#     skip=3,
#     map_genes=False
# )

In [8]:
# for i, (X, y) in enumerate(tqdm(trainloader_map)):
#     if i == 200:
#         break

In [9]:
# for i, (X, y) in enumerate(tqdm(trainloader_nomap)):
#     if i == 200:
#         break
#     X = clean_sample(X, refgenes, train.features)
    

In [10]:
# train_loader, val_loader, test_loader = generate_loaders(
#     datafiles,
#     labelfiles,
#     'Type',
#     num_workers=0,
#     collocate=True,
# )

In [11]:
# for X, y in train_loader:
#     print(type(train_loader))

In [12]:
len(refgenes)

16604

In [13]:
from models.lib.neural import TabNetGeneClassifier

model = TabNetGeneClassifier(
    input_dim=len(refgenes),
    output_dim=19
)

In [14]:
class SampleLoader(torch.utils.data.DataLoader):
    def __init__(self, refgenes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.refgenes = refgenes
        self.currgenes = self.dataset.columns 
            
    def __iter__(self):
        for batch in super().__iter__():
            yield clean_sample(batch[0], self.refgenes, self.currgenes), batch[1]

In [15]:
test = SampleLoader(refgenes=refgenes, dataset=train, batch_size=11, num_workers=0)
next(iter(test))

(tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.9815, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [1.0537, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.7632, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [1.0228, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 tensor([ 6,  4,  4,  4, 16,  4,  4,  4, 16, 16,  4]))

In [16]:
next(iter(test))[0].shape

torch.Size([11, 16604])

In [17]:
from functools import partial 

def custom_collate(sample):
    data = torch.stack([x[0] for x in sample])
    labels = torch.tensor([x[1] for x in sample])
    return data, labels

test = DataLoader(dataset=train, collate_fn=custom_collate, batch_size=4)

## PyTorch-Lightning compatible TabNet architecture 

In [18]:
from models.lib.neural import GeneClassifier
from models.lib.neural import TabNetGeneClassifier
import torch.functional as F

base_model = TabNetGeneClassifier(
    input_dim=len(refgenes),
    output_dim=19,
)

class GeneClassifier(pl.LightningModule):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        self.base_model = TabNetGeneClassifier(
            input_dim=input_dim,
            output_dim=output_dim,
        )
        
    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        out = self.base_model(x)
        return out

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        y_hat = self(x)
        
        loss = F.cross_entropy(y_hat, y)
        # Logging to TensorBoard by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [56]:
base_model.input_dim

16604

In [60]:
classifier = GeneClassifier(
    input_dim=len(refgenes),
    output_dim=19,
)

In [61]:
sample = next(iter(trainloaders[0]))[0]
classifier(sample)

tensor([[-0.8448,  0.7586, -1.3998,  0.0651,  0.2534, -0.6008,  1.3020, -0.5995,
         -1.2490, -1.7024,  0.7147, -0.1489, -0.7904,  1.3752, -0.3794, -0.2655,
         -0.0343,  1.8146,  0.4887],
        [-1.6459,  0.3161, -0.4659,  0.0587, -0.9774,  0.3582,  0.5182,  1.1889,
         -0.3940,  0.8488,  0.5649,  0.2387,  0.6356,  1.2020,  1.1342, -0.2665,
          1.1343,  1.5643, -0.0797],
        [ 0.2180, -0.3618, -0.6595, -0.1235, -0.2284,  0.8975, -0.8446,  1.0732,
          1.2151,  0.0133, -0.6522, -0.1236,  0.8981,  1.2028, -0.8417,  0.0313,
         -0.6274,  1.7479, -0.7256],
        [ 1.7444, -1.4065, -1.0772, -0.4756, -1.4739,  0.6613,  0.8600, -0.9243,
          0.7876,  0.0924,  1.6400, -0.7856, -1.0382,  1.7035, -2.1024, -1.3298,
          0.1758,  4.0664, -1.6334]], grad_fn=<MmBackward0>)

In [66]:
import pytorch_lightning as pl 
from typing import *
import torch.nn.functional as F

from itertools import chain
def custom_collate(sample, refgenes, currgenes):
    data = clean_sample(torch.stack([x[0] for x in sample]), refgenes, currgenes)
    labels = torch.tensor([x[1] for x in sample])
    return data, labels

class CollateLoader(torch.utils.data.DataLoader):
    def __init__(self, refgenes, currgenes, *args, **kwargs):
        collate_fn = functools.partial(custom_collate, refgenes=refgenes, currgenes=currgenes)
        super().__init__(collate_fn = collate_fn, *args, **kwargs)

class SequentialLoader:
    def __init__(self, dataloaders):
        self.dataloaders = dataloaders

    def __len__(self):
        return sum([len(dl) for dl in self.dataloaders])

    def __iter__(self):
        yield from chain(*self.dataloaders)
                
class GeneDataModule(pl.LightningDataModule):
    def __init__(
        self, 
        datafiles: List[str],
        labelfiles: List[str],
        class_label: str,
        refgenes: List[str],
        batch_size: int=16,
        num_workers=32,
        *args,
        **kwargs,
    ):
        super().__init__()
        self.__dict__.update(**kwargs)

        self.datafiles = datafiles
        self.labelfiles = labelfiles
        self.class_label = class_label
        self.refgenes = refgenes
        
        self.num_workers = num_workers
        self.batch_size = batch_size
        
        self.trainloaders = []
        self.valloaders = []
        self.testloaders = []
        
        self.args = args
        self.kwargs = kwargs
        
    def prepare_data(self):
        # Download data from S3 here 
        pass 
    
    def setup(self, stage: Optional[str] = None):
        trainloaders, valloaders, testloaders = generate_loaders(
            datafiles=self.datafiles,
            labelfiles=self.labelfiles,
            class_label=self.class_label,
            refgenes=self.refgenes,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            *self.args,
            **self.kwargs
        )
        
        self.trainloaders = SequentialLoader(trainloaders)
        self.valloaders = SequentialLoader(valloaders)
        self.testloaders = SequentialLoader(testloaders)
        
    def train_dataloader(self):
        return self.trainloaders

    def val_dataloader(self):
        return self.valloaders

    def test_dataloader(self):
        return self.testloaders

In [67]:
s = SequentialLoader(trainloaders)
next(iter(s))

(tensor([[1.1458, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.8902, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 tensor([ 6,  8, 17,  4]))

In [68]:
module = GeneDataModule(
    datafiles=datafiles, 
    labelfiles=labelfiles, 
    class_label='Type', 
    refgenes=refgenes,
    skip=3, 
    normalize=True,
    batch_size=8,
    num_workers=0,
)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

wandb_logger = WandbLogger(name='TabNet Gene Classifier')
trainer = Trainer(logger=wandb_logger)
trainer.fit(classifier, datamodule=module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:116: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(

  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 1.1 M 
----------------------------------------------------
1.1 M     Trainable

Training: 0it [00:00, ?it/s]

Now, we'll subset and define our DataLoaders

In [ ]:
import wandb
from torchmetrics.functional import accuracy

wandb.init()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train_loss = []
val_loss = []
test_loss = []

mod = 10
wandb.watch(model)
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    epoch_loss = 0.0
    # Train loop
    model.train()
    for idx, train in enumerate(train_loader):
        print(f'On loader {idx = }')
        for i, data in enumerate(tqdm(train)):
            print(f'On minibatch {i = }/10')
            if i == 10:
                break 
            inputs, labels = data
            # CLEAN INPUTS
            inputs = clean_sample(inputs, refgenes, train.dataset.columns)
            # Forward pass ➡
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass ⬅
            optimizer.zero_grad()
            loss.backward()

            # Step with optimizer
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_loss += loss.item()


            if i % mod == 0: # record every 2000 mini batches 
                metric_results = calculate_metrics(
                    outputs=outputs,
                    labels=labels,
                    append_str='train',
                    num_classes=model.output_dim,
                    subset='weighted_accuracy',
                )

                wandb.log(metric_results)
                running_loss = running_loss / mod
                wandb.log({f"batch_train_loss": loss})

                running_loss = 0.0
            
    wandb.log({f"epoch_train_loss": epoch_loss / len(train)})
    
    model.eval()
    with torch.no_grad(): # save memory but not computing gradients 
        running_loss = 0.0
        epoch_loss = 0.0
        
        for val in val_loader:
            print(f'On loader {i = }')
            for i, data in enumerate(val):
                if i == 10:
                    break 
                inputs, labels = data
                # CLEAN INPUTS
                inputs = clean_sample(inputs, refgenes, val.dataset.columns)
                # Forward pass ➡
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # print statistics
                running_loss += loss.item()
                epoch_loss += loss.item()

                if i % mod == 0: #every 2000 mini batches 
                    running_loss = running_loss / mod
                    wandb.log({"val_loss": loss})
                    running_loss = 0.0

                    metric_results = calculate_metrics(
                        outputs=outputs,
                        labels=labels,
                        num_classes=model.output_dim,
                        subset='weighted_accuracy',
                        append_str='val',
                    )

                wandb.log(metric_results)
    
        wandb.log({f"epoch_val_loss": epoch_loss / len(train)})


In [ ]:
model

In [ ]:
def test_loop(
    model,
    testloaders,
    refgenes,
    criterion,
    mod,
):
    model.eval()
    
    with torch.no_grad():
        for idx, test in enumerate(testloaders):
            print(f'On {idx = }')
            running_loss = 0.0
            for i, data in enumerate(test):
                print(f'minibatch {i = }')
                if i == 10:
                    break
                inputs, labels = data
                # CLEAN INPUTS
                inputs = clean_sample(inputs, refgenes, test.dataset.columns)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # print statistics
                running_loss += loss.item()
                if i % mod == 0: #every 2000 mini batches 
                    running_loss = running_loss / mod
                    wandb.log({"test_loss": loss})
                    running_loss = 0.0

                    metric_results = calculate_metrics(
                        outputs=outputs,
                        labels=labels,
                        num_classes=model.output_dim,
                        subset='weighted_accuracy',
                        append_str='test',
                    )

                    wandb.log(metric_results)


In [ ]:
test_loop(model, test_loader, refgenes, criterion, mod)

In [ ]:
import matplotlib.pyplot as plt 

plt.plot(train_loss, label='Train')
plt.plot(val_loss, label='Val')
plt.legend()
plt.show()

In [ ]:
labelfiles

In [ ]:
from numpy import memmap

In [ ]:
f = memmap('../data/interim/allen_cortex_T.csv', dtype=np.float64, mode='r')